In [2]:
# Import packages
import ee
import geemap
import os
import json

import time
import subprocess as subp

from osgeo import gdal, gdal_array
from datetime import datetime, timedelta

In [3]:
date_tdy = datetime.today() 
date_tdy = date_tdy.strftime('%Y-%m-%d')
print(date_tdy)

ee.Initialize() # Initialize

f=  open('../../../../../src/data_preprocessing/base_geojson/TL_state_shapefile_for_clip.geojson')
data =json.load(f)

data = data['features'][0]['geometry']['coordinates'][0]
roi = ee.Geometry.Polygon(data)


2023-01-10


In [4]:
collection_subset = ee.ImageCollection("MODIS/061/MOD13Q1") \
    .filterDate('2012-01-01', '2012-01-11').sort('IMAGE_DATE').select('NDVI') # Only select images for the years 2016-2021
print(collection_subset.size().getInfo()) # Shows the number of images within the subcollection
image = collection_subset.first().select('NDVI')  # Pick the first image from the 'list' and select the layer of interest
geemap.image_props(image).getInfo() # Finds basic info of this image

out = os.path.join('GEE_NDVItifs_scaled') # Set path to where we want to save the GeoTIF
# Now export each image within the collection to a GeoTIF
geemap.ee_export_image_collection(collection_subset, out_dir = out, scale=1000, region=roi, file_per_band=True, crs='EPSG:4326')

# image.select('NDVI').projection().nominalScale() sets the scale equal to the resolution of the images within the collection
# file_per_band=False: all bands are downloaded and put as one file
# file_per_band=True: each band is downloaded in a single image

1
Total number of images: 1

Exporting 1/1: 2012_01_01.tif
Generating URL ...
Please wait ...
Data downloaded to /home/luckyw0w/dicra/analytics/sandbox/notebooks/dppd_functions/rasters/GEE_NDVItifs_scaled




In [5]:

basepath='GEE_NDVItifs_scaled/'

if 'projected' not in os.listdir():
    os.mkdir('projected')
else:
    print('directory exists')

arr = os.listdir('GEE_NDVItifs_scaled')
for i in arr:
    cmd="gdalwarp -of GTIFF  -r cubic -t_srs '+proj=longlat +datum=WGS84 +no_defs'"+" "+basepath+str(i)+" projected/"+str(i)
    print(cmd)
    try:
        subp.check_call(str(cmd), shell=True)
    except:
        print("end")
    time.sleep(1)

# Open band 1 as array
arr = os.listdir('projected/')

if 'recalculated' not in os.listdir():
    os.mkdir('recalculated')
else:
    print('directory exists')

for i in arr:
    ds = gdal.Open('projected/'+str(i))
    b1 = ds.GetRasterBand(1)
    arr_q = b1.ReadAsArray()
   # ndv = 1
    # apply scale factpr
    data = arr_q*(0.0001)
    #data = np.where(data > 1,-9999, data)
    #data=np.where(data<0,-9999,data)
    print(str(i))
    # save array, using ds as a prototype
    output="recalculated/"+str(i)
    gdal_array.SaveArray(data.astype("float32"), output, "GTIFF", ds)

    ds = None

basepath='recalculated/'
arr = os.listdir('recalculated')

if 'clipped' not in os.listdir():
    os.mkdir('clipped')
else:
    print('directory exists')
    
for i in arr:
    cmd="gdalwarp -dstnodata -9999 -cutline tsdm/District_Boundary.shp -crop_to_cutline "+basepath+str(i)+" clipped/"+i
    print(cmd)
    try:
        subp.check_call(str(cmd), shell=True)
    except:
        print("end")
    time.sleep(1)

arr = os.listdir('clipped/')

if 'cog_ndvi' not in os.listdir():
    os.mkdir('cog_ndvi')
else:
    print('directory exists')

for i in arr:
    cmd="gdal_translate clipped/"+str(i)+" cog_ndvi/"+str(i)+" -co COMPRESS=LZW -co TILED=YES"
    print(cmd)
    try:
        subp.check_call(str(cmd), shell=True)
    except:
        print("end")
    time.sleep(1)

cmd="rm -r GEE_NDVItifs_scaled clipped projected recalculated"
print(cmd)
try:
    subp.check_call(str(cmd), shell=True)
except:
    print("end")

gdalwarp -of GTIFF  -r cubic -t_srs '+proj=longlat +datum=WGS84 +no_defs' GEE_NDVItifs_scaled/2012_01_01.tif projected/2012_01_01.tif
Creating output file that is 454P x 456L.
Processing GEE_NDVItifs_scaled/2012_01_01.tif [1/1] : 0Using internal nodata values (e.g. -32768) for image GEE_NDVItifs_scaled/2012_01_01.tif.
Copying nodata values from source GEE_NDVItifs_scaled/2012_01_01.tif to destination projected/2012_01_01.tif.
...10...20...30...40...50...60...70...80...90...100 - done.
2012_01_01.tif
gdalwarp -dstnodata -9999 -cutline tsdm/District_Boundary.shp -crop_to_cutline recalculated/2012_01_01.tif clipped/2012_01_01.tif
Creating output file that is 453P x 454L.
Processing recalculated/2012_01_01.tif [1/1] : 0...10...20...30...40...50...60...70...80...90...100 - done.
gdal_translate clipped/2012_01_01.tif cog_ndvi/2012_01_01.tif -co COMPRESS=LZW -co TILED=YES
Input file size is 453, 454
0...10...20...30...40...50...60...70...80...90...100 - done.
rm -r GEE_NDVItifs_scaled clipped